<a href="https://colab.research.google.com/github/femolinas/Projeto_Acha_Preco/blob/main/Projeto_Acha_Preco_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bora Voar? Acha o Preço!**

A primeira instrução realiza as devidas instalações e importações ao projeto.

In [48]:
#Instala software development kit da Google /
!pip install -q -U google-generativeai #instala na shell via pip no modo silencioso (-q) e na versão mais recente (-U) a biblioteca:  google-generativeai

In [49]:
import google.generativeai as genai # Importa/ Carrega a biblioteca da IA Gemini da Google
import pandas as pd
import re
import textwrap   # Importa biblioteca de formato de texto
from IPython.display import display
from IPython.display import Markdown
from google.colab import userdata

Configuração da API Key do google AI Studio (Gemini) e dos parâmetros necessários.

In [50]:
GOOGLE_API_KEY=userdata.get("Secret_Key")
genai.configure(api_key=GOOGLE_API_KEY) #Configuração da chave de API informada

In [51]:
# Set up the model
generation_config = {
  "temperature": 0.9,
  "top_p": 1,
  "top_k": 0,
  "max_output_tokens": 2048,
}

In [52]:
safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]


In [53]:
model = genai.GenerativeModel(model_name="gemini-1.0-pro",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

In [54]:
chat = model.start_chat(history=[]) # guarda o histórico do chat

Pede para o usuário a inserção dos dados necessários e já estabelece o pront para ser enviado a AI.

In [56]:
# O modelo passará a coletar os dados necessários com o usuário
print("Bem vindo ao planejador de viagens mais acessíveis!")

print("\nInforme os seguintes dados da sua viagem:") # Saúda o usuário
origem=input("\nOrigem: ") # Usuário informa a Origem do vôo
destino=input("Destino: ") #Usuário informa o destino do vôo
data_ida=input("Data da Viagem (pode ser uma data exata ou aproximada, mas não esqueça de infomar o mês e o ano!): ") # Data da viagem
retorno=int(input("Informe 1 caso seu vôo seja apenas de ida e 2 para ida e volta: ")) #Vôo apenas de Ida ou Ida e volta

while retorno!=1 and retorno!=2: # Caso haja digitação errada pelo usuário
        retorno=int(input("\n Desculpe, não entendi,poderia repetir? Informe 1 caso seu vôo seja apenas de ida e 2 para ida e volta: "))

if retorno==2: #Ou seja, prompt para o caso de passagem de ida e volta
        prompt_part = "Imagine que você é um passageiro e quer procurar passagens aéreas mais baratas, pesquise o site Google Flights e liste em formato de tabela os dados: data da viagem de ida - horário do vôo de ida - número do vôo de ida - Quantidade de conexões de ida - horário das conexões de ida -  quantidade de bagagens - companhia aerea da ida- data de retorno - horário de retorno- numero de voo de volta - conexões de volta- horario das conexões de volta - quantidade de bagagens de volta- companhia aérea da volta - preço total de uma viagem"
        data_retorno=input("Data do retorno (pode ser uma data exata ou aproximada): ") # Data da viagem
        novo_prompt = f"{prompt_part} de {origem} para {destino} em {data_ida} e retorno em {data_retorno}. O seu resultado deve estar disponível em uma tabela de 15 colunas "
else: #Ou seja, prompt para o caso de passagem de ida apenas
        prompt_part = "Imagine que você é um passageiro e quer procurar passagens aéreas mais baratas, pesquise o site Google Flights e liste em formato de tabela os dados: data da viagem - horário do vôo - número do vôo - Quantidade de conexões - horário das conexões -  quantidade de bagagens - companhia aérea- preço total de uma viagem em uma única tabela"
        novo_prompt = f"{prompt_part} de {origem} para {destino} em {data_ida}"

Bem vindo ao planejador de viagens mais acessíveis!

Informe os seguintes dados da sua viagem:

Origem: São Paulo
Destino: Lisboa
Data da Viagem (pode ser uma data exata ou aproximada, mas não esqueça de infomar o mês e o ano!): novembro de 2024
Informe 1 caso seu vôo seja apenas de ida e 2 para ida e volta: 22

 Desculpe, não entendi,poderia repetir? Informe 1 caso seu vôo seja apenas de ida e 2 para ida e volta: 11

 Desculpe, não entendi,poderia repetir? Informe 1 caso seu vôo seja apenas de ida e 2 para ida e volta: 10000

 Desculpe, não entendi,poderia repetir? Informe 1 caso seu vôo seja apenas de ida e 2 para ida e volta: 2
Data do retorno (pode ser uma data exata ou aproximada): dezembro de 2024


A IA sempre coloca marcadores separando o Cabeçalho e em eventuais testes, esses marcadores geraram erro por estarem em quantidade superior ao número de colunas, portanto o código os une para mais adiante serem removidos

In [57]:
novo_prompt2=str(novo_prompt)
response = chat.send_message(novo_prompt2)
texto=response.text
texto=re.sub(r'\|[-]+\|', '', texto)

Manipulação dos dados para se gerar um data_frame, como em cada teste realizado ocorreram pequenas variações no nome gerado pela IA para as colunas, elas passaram a ser fixadas com base na ordem solicitada para o retorno (prompt)

In [58]:
# Extrai as linhas da resposta
linhas = texto.strip().split("\n")

# Com base na nomenclatura do campo esperado, busca a linha em que começa o cabeçalho
indice1 = None
for i, linha in enumerate(linhas):
    if "| " in linha:
        indice1 = i
        break


# Extrai o cabeçalho
cabecalho = [header.strip() for header in linhas[i].split("|") if header.strip()]


# Extraindo os dados
dados = []
for linha in linhas[(i+1):]:
    colunas = [data.strip() for data in linha.split("|") if data.strip()]
    dados.append(colunas)

# Criando o DataFrame
df = pd.DataFrame(dados, columns=cabecalho)

#Exibe o título
texto_formatado = "<h1><b>Seguem os preços das menores passagens</b></h1>"
display(Markdown(texto_formatado))
print("\n")

if retorno==1: # Garante que o Cabeçalho sempre será o mesmo para viagens de ida
   df.columns = ["data_da_viagem","horario_do_voo", "número_vôo", "conexoes", "horario_conexoes", "quantidade_bagagens", "companhia_aerea","preco"]
elif retorno==2: # Garante que o Cabeçalho sempre será o mesmo para viagens de ida e volta
   df.columns = ["data_da_viagem","horario_do_voo_ida", "número_vôo_ida", "conexoes_ida", "horario_conexoes_ida", "bagagem_ida","companhia_aerea_ida", "data_retorno","horario_retorno","numero_voo_volta","conexoes_volta","horario_conexoes_volta","bagagem_volta","companhia_aerea","preco"]

# Exibindo o DataFrame como uma tabela
df = df.dropna(subset=["preco"]) #limpa os dados referentes a comentários e que por algum motivo estejam sem o preço
df = df[df["data_da_viagem"] != '---'] #limpa a divisória "---"

df #exibe o dataframe


<h1><b>Seguem os preços das menores passagens</b></h1>

,data_da_viagem,horario_do_voo_ida,número_vôo_ida,conexoes_ida,horario_conexoes_ida,bagagem_ida,companhia_aerea_ida,data_retorno,horario_retorno,numero_voo_volta,conexoes_volta,horario_conexoes_volta,bagagem_volta,companhia_aerea,preco
1,01/11/2024,08:00,LATAM 8705,1,"14:00 (Guarulhos, GRU)",1,LATAM,06/12/2024,16:00,TAP 258,1,"20:00 (Lisboa, LIS)",1,TAP,"R$ 3.500,00"
2,03/11/2024,10:00,Azul 2514,0,-,1,Azul,04/12/2024,18:00,TAP 260,1,"22:00 (Guarulhos, GRU)",1,TAP,"R$ 3.600,00"
3,05/11/2024,12:00,Gol 4521,0,-,1,Gol,07/12/2024,14:00,LATAM 8706,1,"18:00 (Guarulhos, GRU)",1,LATAM,"R$ 3.700,00"
4,07/11/2024,14:00,LATAM 8707,0,-,1,LATAM,09/12/2024,16:00,TAP 259,0,-,1,TAP,"R$ 3.800,00"
5,09/11/2024,16:00,Azul 2515,1,"14:00 (Guarulhos, GRU)",1,Azul,11/12/2024,14:00,LATAM 8708,1,"18:00 (Lisboa, LIS)",1,LATAM,"R$ 3.900,00"


Detalha em texto o valor do menor preço e a companhia aérea que realiza o trajeto

In [59]:
df2 = df.copy()
df2["valor_monetario"] = df['preco'].str.replace(',', '.', regex=False).str.extract('(\d+\.?\d*)').astype(float)
index_min=df2["valor_monetario"].idxmin()

valor_minimo = df.loc[index_min,"preco"] #Encontra o menor valor da passagem:

display(Markdown("<b>Menor Valor da Passagem: </b>"))
print(valor_minimo)

if retorno==1:
  display(Markdown("<b>Disponível pela companhia: </b>"))
  Companhia_aerea = df.loc[index_min,"companhia_aerea"] # Retorna a companhia com o menor valor
  print(Companhia_aerea)
elif retorno==2:
  companhia_aerea_ida = df.loc[index_min,"companhia_aerea_ida"] # Retorna a companhia  de ida para o menor valor total
  companhia_aerea_retorno = df.loc[index_min,"companhia_aerea"] # Retorna a companhia de volta para o menor valor total
  display(Markdown("<b>Ida disponível pela companhia: </b>"))
  print(companhia_aerea_ida)
  display(Markdown("<b>Retorno disponível pela companhia: </b>"))
  print(companhia_aerea_retorno)

<b>Menor Valor da Passagem: </b>

R$ 3.500,00


<b>Ida disponível pela companhia: </b>

LATAM


<b>Retorno disponível pela companhia: </b>

TAP


O Nosso Projeto passa então com base nos dados inseridos a sugerir eventos culturais para o usuário

In [60]:
#Exibe o título
texto_formatado2 = f"<h1><b>Enquanto estiver em {destino} seguem algumas sugestões: </b></h1>"
display(Markdown(texto_formatado2))
print("\n")
prompt = f"imagine que você é um entusiasta por atividades culturais, descreva em tópicos algumas atividades e locais históricos a serem visitados em {destino}, não esqueça de sugerir pratos típicos. Não separe por dias (pode ser que não coincida com os planos do cliente)"

response2 = chat.send_message(str(prompt))

display(Markdown(response2.text))




<h1><b>Enquanto estiver em Lisboa seguem algumas sugestões: </b></h1>

**Atividades Culturais e Locais Históricos em Lisboa:**

* **Visitar o Castelo de São Jorge:** Explore a antiga fortaleza mourisca que oferece vistas panorâmicas da cidade.
* **Passear pelo Alfama:** Percorra as ruas estreitas e íngremes do bairro mais antigo de Lisboa, conhecido por suas casas coloridas e fado ao vivo.
* **Explorar a Baixa Pombalina:** Admire a arquitetura neoclássica do centro histórico reconstruído após o terremoto de 1755.
* **Visitar o Mosteiro dos Jerónimos:** Maravilhe-se com a arquitetura manuelina deste Patrimônio Mundial da UNESCO, lar do túmulo de Vasco da Gama.
* **Descobrir o Museu Nacional do Azulejo:** Aprenda sobre a história e a arte dos azulejos portugueses nesta coleção abrangente.
* **Assistir a um espetáculo de fado:** Experimente a música tradicional portuguesa em um dos muitos locais de fado da cidade.
* **Visitar o Museu Nacional de Arte Antiga:** Admire obras-primas da arte portuguesa e europeia, incluindo pinturas, esculturas e joias.
* **Explorar o Palácio da Ajuda:** Visite a antiga residência real e seus belos jardins.
* **Passear pelo Parque das Nações:** Descubra a área moderna construída para a Exposição Mundial de 1998, com museus, restaurantes e espaços verdes.

**Pratos Típicos:**

* **Bacalhau à Gomes de Sá:** Bacalhau salgado refogado com batatas, cebolas e ovos.
* **Sardinhas Assadas:** Sardinhas grelhadas servidas com batatas cozidas ou salada.
* **Caldo Verde:** Sopa tradicional feita com couve galega, batatas e chouriço.
* **Pastéis de Nata:** Tortas de creme de ovo folhadas.
* **Arroz de Marisco:** Arroz cozido com vários tipos de frutos do mar.